In [3]:
import pyrankvote
import ast
import pandas as pd
import numpy as np
from pyrankvote import Candidate, Ballot
import glob
import os


def create_master_file(file_name_pattern):
    file_names = [name for name in glob.glob(file_name_pattern)]
    # combine all files in the list
    data = []
    for csv in file_names:
#         print('csv:', csv)
        frame = pd.read_csv(csv)
#         print("before:", len(frame))
        frame['filename'] = os.path.basename(csv)
        filter_col = [col for col in frame if col.startswith('candidate_')]
        for col in filter_col:
            idx = frame[col].str.isnumeric() & (frame[:][col] != '0')
            frame = frame[~idx]
#             print("after:", len(frame))
        data.append(frame)
    # concat all dfs
    combined_csv_df = pd.concat(data)
    # export to csv
#     combined_csv_df.to_csv(master_file_path, index=False)
    return combined_csv_df
    
    
def get_cand_list(df):
    list_of_unique_vals_in_dataframe = df.iloc[:, 1].value_counts().index
    cand_list = list(filter(lambda x: x.startswith('can'), list_of_unique_vals_in_dataframe))
    return cand_list


def initialize_cand_objs(cand_list):
    if isinstance(cand_list, str):
        return [Candidate(c) for c in cand_list.split(', ')]
    else:
        return [Candidate(c) for c in cand_list]


def initialize_cand_objs_in_df_cols(df):
    cands_df = df.iloc[:, 1:]  # getting all columns with candidate names in them
    cands_df = cands_df.applymap(lambda x: create_cand(x))  # making them into pyrankvote candidate objects
    cands_df['ballot_id'] = df.iloc[:, 0]  # stitching df back with ballot_ids from old dataframe
    return cands_df 

def get_cands_into_single_cell(df):
    df['candidate_list'] = df.iloc[:,].agg(", ".join, axis=1) #this excludes the first candidates??
    return df

def initialize_ballot_objs(df):
    ballot_objects = []
    for index, value in enumerate(df['candidate_list']):
        ballot = Ballot(ranked_candidates=value)
        ballot_objects.append(ballot)
    return ballot_objects


def run_election(list_of_cand_objs, election_df):
    return pyrankvote.instant_runoff_voting(list_of_cand_objs, election_df['ballots'], pick_random_if_blank=True)

def rm_invalid_rows(df):
    return df[df['candidate_list'] != '0']


def pyrank_main(file_name_pattern = "../data/*.csv"):    
    
    df=create_master_file(file_name_pattern) #step 1: df

    election_dict = {}   # step 2: create dictionary
    
    df.rename(columns={'Unnamed: 0': 'ballot_id'}, inplace=True) # should be apart of the generating and concatenation script   
    
    #getting  number of candidates:
    num_cands = df['num_candidates'].max()
    print("number of candidates in election:", num_cands)
    election_dict['num_candidates'] = num_cands   
    
    #droping nans, 
    df = df.dropna(axis=1, how='all')
    df = df.drop(['ballot_id','filename','noise','num_candidates'], axis=1)
    print('df.shape',df.shape)    #aggregating all cand into one cell
    
    df = get_cands_into_single_cell(df)    #Removing 0 with empty space:
    df['candidate_list'] = df['candidate_list'].apply(lambda x: x.replace('0, ', '').replace(', 0', ''))    #Removing invalid rows:
    df = rm_invalid_rows(df)    #for all srings making into pyrank candidate objects:
    df['candidate_list'] = df['candidate_list'].apply(lambda x: initialize_cand_objs(x))    #creating ballot objects column for each candidate in candidate list:
    ballots = initialize_ballot_objs(df)
    df['ballots'] = ballots    #getting unique list of candidates:
    cand_list = get_cand_list(df)
    cand_list = initialize_cand_objs(cand_list)
    
    print("cand_list: ",cand_list)    #running pyrank election and getting winners:
    election = run_election(cand_list, df)
    winner = election.get_winners()    #Dictionary keys and values *************************
    #printing winners and add dictionary key:    print("get_winners",winner)
    election_dict['winner'] = winner    # printing rounds and add dictionary key:
    print("rounds",len(election.rounds))
    
    election_dict['rounds'] = len(election.rounds)    #print elections and add dictionary key:
    print("election:", election)
    
    election_dict['election'] = election    #will return df and dictionary for us to read votes
    
    return df, election_dict


In [2]:
# file_name_pattern = "../data/*.csv"
# master_file_path = "master_election_new.csv"
# create_master_file(file_name_pattern, master_file_path)
# df = pd.read_csv("master_election_new.csv")

master_df = df.copy(deep=True)
election_ids = df.filename.unique()

election_num_dict = {}
for election_num, election in enumerate(election_ids):
    election_num_dict[election_num] = election

NameError: name 'df' is not defined

In [3]:
#step 4: 

def read_votes(file, election_dict):
    with open(file, 'r') as file1:
        count = 0
        while True: 
            count += 1

            # Get next line from file 
            line = file1.readline()
            if not line: #if line doesn't exist, break
                break
            strip_line = line.strip() 
            
            #for n candidates per election there will always be n-1 rounds
            num_cands = election_dict['num_candidates']
            rounds = election_dict['rounds']
            winner = election_dict['winner']

            for i in range(1, num_cands+1):
                for j in range(1, rounds):
                    if 'ROUND' in strip_line or "FINAL" in strip_line:
                        start_round = True
                        if "ROUND {}".format(j) in strip_line:
                            monitor_round = True
                        else:
                            monitor_round=False
                        if "FINAL" in strip_line:
                            final_round = True
                        else:
                            final_round = False    
                            
                            
                    if 'ROUND' not in strip_line and 'FINAL' not in strip_line:
                        start_round = False
                        
                        if 'candidate_{}'.format(i) in line and monitor_round:
                            if 'can{0}{1}RoundVotes'.format(i, j) not in election_dict:
                                election_dict['can{0}{1}RoundVotes'.format(i, j)]=[int(s) for s in strip_line.split() if s.isdigit()].pop()

                        if 'candidate_{}'.format(i) in line and final_round:
                            if 'can{0}{1}RoundVotes'.format(i, 'final') not in election_dict:
                                election_dict['can{0}{1}RoundVotes'.format(i, 'final')]=[int(s) for s in strip_line.split() if s.isdigit()].pop()

                        if winner in line and monitor_round:
                            if 'winner{}RoundVotes'.format(j) not in election_dict:
                                election_dict['winner{}RoundVotes'.format(j)]=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                                
                        if winner in line and final_round:
                            if 'winner{}RoundVotes'.format('final') not in election_dict:
                                election_dict['winner{}RoundVotes'.format('final')]=[int(s) for s in strip_line.split() if s.isdigit()].pop()

        file1.close()    



# '{0} in {1}'.format(unicode(self.author,'utf-8'),  unicode(self.publication,'utf-8'))

In [4]:
#Step 3:

#building dictionary
all_elect_dict={}


#Enumerating through the election and election_num in the election_id array and applying main function while printing:
for election_num, election in enumerate(election_ids):
    
    print(election_num, election,' \n ')
    print("***********Election: ", election_num)
    df_election = df.loc[df['filename']==election]


    temp_df,election_dict = main(df_election)
    print(election_dict)


    if len(election_dict['winner'])==1:
        winner = election_dict['winner'][0].name
        election_dict['winner'] = winner
    else:
        print(election, "has more than one winner")
    result_file = 'out.txt'

    with open(result_file, 'w') as file1:
        print(election_dict['election'], file=file1)
    read_votes(result_file, election_dict) #<<<==== where we actually call read_votes
    all_elect_dict[election_num] = election_dict


0 election_07-21-2020_10-40-22_3cands_0.006666666666666667noise.csv  
 
***********Election:  0
number of candidates in election: 3
df.shape (41673, 3)


c:\users\anxhe\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


cand_list:  [<Candidate('candidate_2')>, <Candidate('candidate_1')>, <Candidate('candidate_3')>]
rounds 2
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_3    18581  Hopeful
candidate_2    13659  Hopeful
candidate_1     9433  Rejected

FINAL RESULT
Candidate      Votes  Status
-----------  -------  --------
candidate_2    21420  Elected
candidate_3    20253  Rejected
candidate_1        0  Rejected

{'num_candidates': 3, 'winner': [<Candidate('candidate_2')>], 'rounds': 2, 'election': <ElectionResults(2 rounds)>}
1 election_07-21-2020_10-40-23_3cands_0.02666666666666667noise.csv  
 
***********Election:  1
number of candidates in election: 3
df.shape (26726, 3)
cand_list:  [<Candidate('candidate_1')>, <Candidate('candidate_2')>, <Candidate('candidate_3')>]
rounds 2
election: ROUND 1
Candidate      Votes  Status
-----------  -------  --------
candidate_2    10359  Hopeful
candidate_3     9727  Hopeful
candidate_1     6640  Rejected

FINAL RESULT
Ca

In [7]:
all_elect_dict

{0: {'num_candidates': 3,
  'winner': 'candidate_2',
  'rounds': 2,
  'election': <ElectionResults(2 rounds)>,
  'can31RoundVotes': 18581,
  'winner1RoundVotes': 13659,
  'can21RoundVotes': 13659,
  'can11RoundVotes': 9433,
  'winnerfinalRoundVotes': 21420,
  'can2finalRoundVotes': 21420,
  'can3finalRoundVotes': 20253,
  'can1finalRoundVotes': 0},
 1: {'num_candidates': 3,
  'winner': 'candidate_3',
  'rounds': 2,
  'election': <ElectionResults(2 rounds)>,
  'can21RoundVotes': 10359,
  'winner1RoundVotes': 9727,
  'can31RoundVotes': 9727,
  'can11RoundVotes': 6640,
  'winnerfinalRoundVotes': 14519,
  'can3finalRoundVotes': 14519,
  'can2finalRoundVotes': 12207,
  'can1finalRoundVotes': 0},
 2: {'num_candidates': 3,
  'winner': 'candidate_2',
  'rounds': 1,
  'election': <ElectionResults(1 rounds)>},
 3: {'num_candidates': 3,
  'winner': 'candidate_3',
  'rounds': 1,
  'election': <ElectionResults(1 rounds)>},
 4: {'num_candidates': 3,
  'winner': 'candidate_3',
  'rounds': 2,
  'elect

In [9]:
#Step 5:

election_results_df=pd.DataFrame()
election_results_list=[]

for key,val in all_elect_dict.items():
    val['election_id']=election_num_dict[key]
    print(val)

    election_results_list.append(val)

In [10]:
# election_results_df = pd.DataFrame(election_results_list)
# election_results_df.to_csv('election_results.csv', index=False)

In [11]:
# election_results_df.head()

,num_candidates,winner,rounds,election,can31RoundVotes,winner1RoundVotes,can21RoundVotes,can11RoundVotes,winnerfinalRoundVotes,can2finalRoundVotes,...,can34RoundVotes,can14RoundVotes,can44RoundVotes,can24RoundVotes,can54RoundVotes,can61RoundVotes,can62RoundVotes,can63RoundVotes,can64RoundVotes,can6finalRoundVotes
0,3,candidate_2,2,ROUND 1\nCandidate Votes Status\n-------...,18581.0,13659.0,13659.0,9433.0,21420.0,21420.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,candidate_3,2,ROUND 1\nCandidate Votes Status\n-------...,9727.0,9727.0,10359.0,6640.0,14519.0,12207.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,candidate_2,1,FINAL RESULT\nCandidate Votes Status\n--...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,candidate_3,1,FINAL RESULT\nCandidate Votes Status\n--...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,candidate_3,2,ROUND 1\nCandidate Votes Status\n-------...,6489.0,6489.0,6907.0,3824.0,9400.0,7820.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#Step 7:
election_results_df.fillna(0, inplace=True)

df = election_results_df 
# df.to_csv("df.csv")
# df = pd.read_csv('df.csv')

In [13]:
df.head()

,Unnamed: 0,num_candidates,winner,rounds,election,can31RoundVotes,winner1RoundVotes,can21RoundVotes,can11RoundVotes,winnerfinalRoundVotes,...,can34RoundVotes,can14RoundVotes,can44RoundVotes,can24RoundVotes,can54RoundVotes,can61RoundVotes,can62RoundVotes,can63RoundVotes,can64RoundVotes,can6finalRoundVotes
0,0,3,candidate_2,2,ROUND 1\nCandidate Votes Status\n-------...,18581.0,13659.0,13659.0,9433.0,21420.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3,candidate_3,2,ROUND 1\nCandidate Votes Status\n-------...,9727.0,9727.0,10359.0,6640.0,14519.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,candidate_2,1,FINAL RESULT\nCandidate Votes Status\n--...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,candidate_3,1,FINAL RESULT\nCandidate Votes Status\n--...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,3,candidate_3,2,ROUND 1\nCandidate Votes Status\n-------...,6489.0,6489.0,6907.0,3824.0,9400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# df.columns

Index(['Unnamed: 0', 'num_candidates', 'winner', 'rounds', 'election',
       'can31RoundVotes', 'winner1RoundVotes', 'can21RoundVotes',
       'can11RoundVotes', 'winnerfinalRoundVotes', 'can2finalRoundVotes',
       'can3finalRoundVotes', 'can1finalRoundVotes', 'election_id',
       'winner2RoundVotes', 'winner3RoundVotes', 'can32RoundVotes',
       'can33RoundVotes', 'can51RoundVotes', 'can52RoundVotes',
       'can53RoundVotes', 'can41RoundVotes', 'can42RoundVotes',
       'can43RoundVotes', 'can22RoundVotes', 'can23RoundVotes',
       'can12RoundVotes', 'can13RoundVotes', 'can5finalRoundVotes',
       'can4finalRoundVotes', 'winner4RoundVotes', 'can34RoundVotes',
       'can14RoundVotes', 'can44RoundVotes', 'can24RoundVotes',
       'can54RoundVotes', 'can61RoundVotes', 'can62RoundVotes',
       'can63RoundVotes', 'can64RoundVotes', 'can6finalRoundVotes'],
      dtype='object')

In [15]:
#getting totals per candidates

#Step 8:
def cand_total(df):
    for j in range(1, df.num_candidates.max()+1):
        df['Can{}Total'.format(j)] =  df.apply(lambda row: row[[col for col in df if col.startswith('can{}'.format(j))]].sum(), axis=1)
    return df
df=cand_total(df)

In [16]:
# df.head()

,Unnamed: 0,num_candidates,winner,rounds,election,can31RoundVotes,winner1RoundVotes,can21RoundVotes,can11RoundVotes,winnerfinalRoundVotes,...,can62RoundVotes,can63RoundVotes,can64RoundVotes,can6finalRoundVotes,Can1Total,Can2Total,Can3Total,Can4Total,Can5Total,Can6Total
0,0,3,candidate_2,2,ROUND 1\nCandidate Votes Status\n-------...,18581.0,13659.0,13659.0,9433.0,21420.0,...,0.0,0.0,0.0,0.0,9433.0,35079.0,38834.0,0.0,0.0,0.0
1,1,3,candidate_3,2,ROUND 1\nCandidate Votes Status\n-------...,9727.0,9727.0,10359.0,6640.0,14519.0,...,0.0,0.0,0.0,0.0,6640.0,22566.0,24246.0,0.0,0.0,0.0
2,2,3,candidate_2,1,FINAL RESULT\nCandidate Votes Status\n--...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,candidate_3,1,FINAL RESULT\nCandidate Votes Status\n--...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,3,candidate_3,2,ROUND 1\nCandidate Votes Status\n-------...,6489.0,6489.0,6907.0,3824.0,9400.0,...,0.0,0.0,0.0,0.0,3824.0,14727.0,15889.0,0.0,0.0,0.0


In [15]:
#Step 9:

# getting lists of votes for each round. was trying to make iterative but having issues so leaving hard coded for now...
def list_rounds(df):
    df['first_rounds_list'] = df[['can11RoundVotes', 'can21RoundVotes', 'can31RoundVotes','can41RoundVotes']].values.tolist()
    df['sec_rounds_list'] = df[['can12RoundVotes', 'can22RoundVotes', 'can32RoundVotes', 'can42RoundVotes']].values.tolist()
    df['third_rounds_list'] = df[['can13RoundVotes', 'can23RoundVotes','can33RoundVotes','can43RoundVotes']].values.tolist()
    df['fourth_rounds_list'] = df[['can14RoundVotes','can24RoundVotes', 'can34RoundVotes', 'can44RoundVotes']].values.tolist()
    df['final_rounds_list'] = df[['can1finalRoundVotes', 'can2finalRoundVotes', 'can3finalRoundVotes', 'can4finalRoundVotes']].values.tolist()
    return df

# for j in range(1, df.num_candidates.max()+1):
#     for i in range(1, df.rounds.max()+1):
#         df['{}_rounds_list'.format(i)] = df[[col for col in df if col.startswith('can{0}{1}'.format(j,i))]].values.tolist()
df= list_rounds(df)

In [16]:
df.head()

,Unnamed: 0,num_candidates,winner,rounds,election,winner1RoundVotes,can21RoundVotes,can11RoundVotes,can31RoundVotes,winnerfinalRoundVotes,...,Can2Total,Can3Total,Can4Total,Can5Total,Can6Total,first_rounds_list,sec_rounds_list,third_rounds_list,fourth_rounds_list,final_rounds_list
0,0,3,candidate_2,2,ROUND 1\nCandidate Votes Status\n-------...,16966.0,16966.0,16376.0,8053.0,21031.0,...,37997.0,8053.0,0.0,0.0,0.0,"[16376.0, 16966.0, 8053.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[20364.0, 21031.0, 0.0, 0.0]"
1,1,3,candidate_1,2,ROUND 1\nCandidate Votes Status\n-------...,9021.0,8531.0,9021.0,8462.0,13230.0,...,21315.0,8462.0,0.0,0.0,0.0,"[9021.0, 8531.0, 8462.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[13230.0, 12784.0, 0.0, 0.0]"
2,2,3,candidate_1,1,FINAL RESULT\nCandidate Votes Status\n--...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]"
3,3,3,candidate_2,1,FINAL RESULT\nCandidate Votes Status\n--...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]"
4,4,3,candidate_1,2,ROUND 1\nCandidate Votes Status\n-------...,6986.0,2448.0,6986.0,6986.0,8244.0,...,2448.0,15162.0,0.0,0.0,0.0,"[6986.0, 2448.0, 6986.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[8244.0, 0.0, 8176.0, 0.0]"


In [17]:
# df.winner1RoundVotes=df.winner1RoundVotes.astype(float)
# df.winner2RoundVotes=df.winner2RoundVotes.astype(float)
# df.winner3RoundVotes=df.winner3RoundVotes.astype(float)
# df.winner4RoundVotes=df.winner4RoundVotes.astype(float)
# df.winnerfinalRoundVotes=df.winnerfinalRoundVotes.astype(float)

#Step 9 (combine with above)

# changing datatype to float:
for i in range(1, df.rounds.max()-1):
    df['winner{}RoundVotes'.format(i)]=df['winner{}RoundVotes'.format(i)].astype(float)
    df.winnerfinalRoundVotes=df.winnerfinalRoundVotes.astype(float)

In [18]:
#Step 10:

def get_first_place(df):
    #grabbing first round votes list for winner first round extracting first element from the rounds votes....
    df['winner_1st_place_votes_r1'] = np.where(df.winner1RoundVotes == df.first_rounds_list.map(lambda x: x[0]),
                                               df.winner1RoundVotes, 0)

    df['winner_1st_place_votes_r2'] = np.where(df.winner2RoundVotes == df.sec_rounds_list.map(lambda x: x[0]),
                                               df.winner2RoundVotes, 0)

    df['winner_1st_place_votes_r3'] = np.where(df.winner3RoundVotes == df.third_rounds_list.map(lambda x: x[0]),
                                               df.winner3RoundVotes, 0)

    df['winner_1st_place_votes_r4'] = np.where(df.winner4RoundVotes == df.fourth_rounds_list.map(lambda x: x[0]),
                                               df.winner4RoundVotes, 0)

    df['total_1st_place_winner_votes'] = df['winner_1st_place_votes_r1'] + df['winner_1st_place_votes_r2'] + df['winner_1st_place_votes_r3'] + df['winner_1st_place_votes_r4']
    return df

def get_last_place(df):

    df['winner_last_place_votes_r1'] = np.where(df.winner1RoundVotes == df.first_rounds_list.map(lambda x: x[-1]),
                                               df.winner1RoundVotes, 0)

    df['winner_last_place_votes_r2'] = np.where(df.winner2RoundVotes == df.sec_rounds_list.map(lambda x: x[-1]),
                                               df.winner2RoundVotes, 0)

    df['winner_last_place_votes_r3'] = np.where(df.winner3RoundVotes == df.third_rounds_list.map(lambda x: x[-1]),
                                               df.winner3RoundVotes, 0)

    df['winner_last_place_votes_r4'] = np.where(df.winner4RoundVotes == df.fourth_rounds_list.map(lambda x: x[-1]),
                                               df.winner4RoundVotes, 0)

    df['total_last_place_winner_votes'] = df['winner_last_place_votes_r1'] + df['winner_last_place_votes_r2'] + df['winner_last_place_votes_r3'] + df['winner_last_place_votes_r4']
    return df
df=get_first_place(df)
df=get_last_place(df)

In [20]:
df.tail()

,Unnamed: 0,num_candidates,winner,rounds,election,winner1RoundVotes,can21RoundVotes,can11RoundVotes,can31RoundVotes,winnerfinalRoundVotes,...,winner_1st_place_votes_r1,winner_1st_place_votes_r2,winner_1st_place_votes_r3,winner_1st_place_votes_r4,total_1st_place_winner_votes,winner_last_place_votes_r1,winner_last_place_votes_r2,winner_last_place_votes_r3,winner_last_place_votes_r4,total_last_place_winner_votes
15,15,3,candidate_2,2,ROUND 1\nCandidate Votes Status\n-------...,8490.0,8490.0,4092.0,6447.0,10553.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,16,3,candidate_3,2,ROUND 1\nCandidate Votes Status\n-------...,18926.0,14562.0,9369.0,18926.0,23578.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,17,6,candidate_5,5,ROUND 1\nCandidate Votes Status\n-------...,16493.0,0.0,15278.0,0.0,24036.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,18,4,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,8273.0,9491.0,8273.0,0.0,14093.0,...,8273.0,8273.0,0.0,0.0,16546.0,0.0,0.0,0.0,0.0,0.0
19,19,4,candidate_1,3,ROUND 1\nCandidate Votes Status\n-------...,13387.0,11212.0,13387.0,0.0,19456.0,...,13387.0,13387.0,0.0,0.0,26774.0,0.0,0.0,0.0,0.0,0.0


In [21]:


df_final = df[[ 'winner', 'rounds', 'election', 'Can1Total', 'Can2Total','Can3Total','Can4Total', 'total_1st_place_winner_votes','total_last_place_winner_votes']]

In [22]:
df_final.to_csv("df_final.csv")